In [1]:
from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()

from bokeh.layouts import widgetbox, column
from bokeh.models import Slider, Label

import numpy as np
np.random.seed(23)

Loading BokehJS ...

# Foundations of Probability

Probability is the foundation of many of the methods and models in data science. Statistical inference methods use data to understand the underlying processes behind the data. With probability we can study the regularities that arise in random phenomena.

We will start simulating in Python and learn to calculate probabilities and work with probability distributions.

# Flipping a coin

Let's try to gain some intutition about probability by starting with the classic random experiment, a coin flip.

If we flip a coin there are two possible outcomes: heads or tails. This random experiment is called a <b>Bernoulli trial</b>, after Jacob Bernoulli. 

<b>A Bernoulli trial is one where the possible outcomes are binary. Each outcome is called an event. We assign probabilities to events.</b>

With a fair coin we have a 50% chance of getting heads and a 50% chance of getting tails for each event.

To simulate the coin flips we can use the <code>bernoulli</code> object from the <code>scipy</code> module where:

- <code>p</code> is the probability of success
-  <code>size</code> is the number of trials

In [2]:
from scipy.stats import bernoulli

# flip the coin
bernoulli.rvs(p=0.5, size=10)

array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0])

# Binomial Distribution

<b>A sequence of independent Bernoulli trial follows the binomial distribution.</b>So, if we want to count the number of successes instead of using the bernoulli object and adding the outcomes, we can use the binomial distribution and the <code>binom</code> object.

In [76]:
from scipy.stats import binom

# count the number of heads 
binom.rvs(n=10, p=0.5, size=5)

array([8, 6, 5, 4, 5])

We used three arguments:

- <code>n</code>: number of flips per trial
- <code>p</code>: probability of success
- <code>size</code>: number of draws of the same experiment

In [4]:
# count the number of heads in 5 experiments
binom.rvs(n=10, p=0.5, size=5)

array([7, 7, 4, 5, 8])

## <b>Probability mass function (pmf)</b>
  
A discrete random variable has a finite number of possible outcomes. The probability mass function allows us to calculate the probability of getting a particular outcome for a discrete random variable.
 
<b>The binomial probability mass function allows us to calculate the probability of getting $k$ heads from $n$ coin flips with $p$ probability.</b>
 
$$binom.pmf(k, n, p) = \binom{n}{k}p^k(1 - p)^{n-k} $$

where

$$\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

The formula multiplies the number of different ways that we can get $k$ successes out of $n$ coin flips by the probability of success $p$ raised by the number of successes $k$ by the probability of failure $1-p$ raised to the number of failures $n-k$.

In [5]:
# probability of 5 heads after 10 throws with a fair coin
binom.pmf(k=5, n=10, p=0.5)

0.24609375000000025

We can plot this to see how the most likely outcome vary.

In [65]:
def plot_pmf(n=10, p=0.5):
    
    # create a figure object
    plot = figure(x_axis_label='number of successes', y_axis_label='probability of outcome', plot_height=400, plot_width=550)

    # define an array with the number of successes we could get
    successes = np.arange(n+1)

    # create an empty list to store probabilities
    probs = []

    # loop over all outcomes
    for k in successes:
    
        # get the probability of the outcome
        y =  binom.pmf(k=k, n=n, p=p)
    
        # plot on figure
        plot.segment(k, 0, k, y, line_width=2)
        plot.circle(k, y, size=16)
        
        # add the probability to our list
        probs.append(y)
    
    #change axis ticks
    plot.xaxis.ticker = successes

    show(plot)
    return plot

_ = plot_pmf(n=20, p=0.5)

## <b>Probability distribution function (cdf)</b>

When we want to calculate the probability of getting $k$ or fewer heads from $n$ throws, we use the binomial probabilty distribution function, which adds the probabilities of getting 0 heads out of $n$ flips, getting 1 head out of 10 flips and so on up to $k$ heads out of $n$ flips.

$$binom.cdf(k, n, p) = \binom{n}{0}p^0(1 - p)^{n} + \binom{n}{1}p(1 - p)^{n-1} + ... + \binom{n}{k}p^k(1 - p)^{n-k}$$

The binomial probability distribution function allows us to calculate the cumulative probability of getting $k$ heads or fewer from $n$ coin flips with $p$ probability of getting heads.

In [8]:
# probability of less than 2 heads after 10 throws with a fair coin
binom.cdf(k=2, n=10, p=0.5)

0.054687500000000014

We can use the complement to see what the probabilities of getting more than $k$ heads from $n$ throws. 

In [9]:
# probability of more than 2 heads after 10 throws with a fair coin
1 - binom.cdf(k=2, n=10, p=0.5)

0.9453125

Alternatively we can use the function <code>binom.sf(k,n,p)</code> for the complement. <code>sf</code> stands for survival function.

In [10]:
# probability of more than 2 heads after 10 throws with a fair coin using binom.sf
binom.sf(k=2, n=10, p=0.5)

0.9453125

In [11]:
def plot_cdf(n=10, p=0.5):
    # create a figure object
    plot = figure(x_axis_label='number of heads', y_axis_label='cdf', plot_height=400, plot_width=550)

    # defining an array with the number of successes we could get
    successes = np.arange(n+1)

    # create a list of probability outcomes
    probs = [binom.cdf(k=k, n=n, p=p) for k in successes]

    # plot cdf
    plot.line(successes, probs, line_width=4, legend='p='+str(p))

    # change axis ticks
    plot.xaxis.ticker = successes
    plot.legend.location = 'bottom_right'
    show(plot)
    return plot

_ = plot_cdf(n=20, p=0.5)

In [12]:
binom.sf(k=4, n=20, p=0.5)

0.9940910339355469

## Expected value

A discrete random variable has finite outcomes. For discrete random variables, the <b>expected value</b> is the sum of the possible outcomes weighted by their probability. To calculate the expected value, each outcome is multiplied by its probability, and the products are summed.

$$E(X)=\sum_{i=1}^{k}x_1p_1 = x_1p_1 + x_2p_2 + ... + x_kp_k$$

It is a value were the probability will concentrate when we repeat the experiment. For our coin example we multiply the tails outcome $0$ for its probability $1-p$ then we add the heads outcome which is $1$ multiplied by its probability $p$

$$E(X)=\sum_{i=1}^{2}x_1p_1 =0\times(1-p) + 1\times p=p$$

and we get the expected value $p$.

## Arithmetic Mean

The <b>arithmetic mean</b> is the sum of each outcome divided by the number of samples.

$$\bar{X}=\frac{1}{n}\sum_{i=1}^{n}x_1=\frac{1}{n}(x_1+x_2+...+x_n)$$

The <b>arithmetic mean</b> of the oucomes converges to the <b>expected value</b> as we increase the number of random experiments. In scipy we will use the <code>describe()</code> function from <code>scipy.stats</code> to get the mean.

In [28]:
from scipy.stats import describe

describe([0,1])

DescribeResult(nobs=2, minmax=(0, 1), mean=0.5, variance=0.5, skewness=0.0, kurtosis=-2.0)

If we repeat the experiment and make $n$ coin flips we should get a number near the expected value, so the more the flips, the more the mean of all the throws approaches the expected value as we can see from the graph below.

In [81]:
def plot_mean_of_flips(p=0.5, size=200):
    
    #generate x values
    x = np.arange(size)
    
    # generate coin flips
    results = binom.rvs(n=1, p=p, size=size)
    
    # get the list of accumulated means
    y = [describe(results[:i+1]).mean for i in x]
    
    # creating the plot
    plot = figure(y_range=(0,1), x_axis_label='Coin flips', y_axis_label='Mean of coin flips', plot_height=400, plot_width=550)
    plot.line(x, y)
    plot.line(x, p, color='grey', line_alpha=0.8)
    label = Label(x=size*3/5, y=p, text="expected value", text_baseline="bottom", text_alpha=0.8)
    plot.add_layout(label)

    show(plot)
    return plot

_ = plot_mean_of_flips(p=0.5, size=1000)

That is the relationship between the expected value and the mean. 

## Variance

The <b>variance</b> measures how concentrated or spread out from the expected value the data is.

$$Var(X)=E[(X-E(X))^2]=\sum_{i=1}^{n}p_i\times (x_i-E(X))^2$$

Variance is the expected value of the squared deviation of a random variable from its expected value. We will again use the <code>describe</code> function and take the variance from the result.

In the particular case of the binomial distribution, the expected value is the product of the number of coin flips and the probability of getting heads.

For $X\sim Binomial(n,p)$
$$E(X)=n\times p$$
$$Var(X)=n\times p \times (1-p)$$

The variance is the product of the expected value and the probability of failure. We can use <code>binom.stats()</code> method to get the expected value and variance of a binomial distribution.

In [71]:
binom.stats(n=1, p=0.5)

(array(0.5), array(0.25))

Using <code>binom.stats()</code> we can make some calculations so we know what we can expect in our simulations. 

Expected value, mean and variance are essential to probability and statistics. In fact these are the most important measures to calculate to determine if the data is spread out or concentrated around the expected value.

In [85]:
averages = []
variances = []

for i in range(0, 1500):
	# 10 draws of 10 coin flips with 25% probability of heads
    sample = binom.rvs(n=10, p=0.25, size=10)
	# Mean and variance of the values in the sample variable
    averages.append(describe(sample).mean)
    variances.append(describe(sample).variance)
  
 # Calculate the mean of the averages variable
print("Mean {}".format(describe(averages).mean))

# Calculate the mean of the variances variable
print("Variance {}".format(describe(variances).mean))

# Calculate the mean and variance
print(binom.stats(n=10, p=0.25))

Mean 2.5048000000000004
Variance 1.896577777777778
(array(2.5), array(1.875))
